# Install BioPython

In [3]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.5 MB/s eta 0:00:00


# Import Modules

In [4]:
from Bio import SeqIO
from Bio.Align import PairwiseAligner

# Load Sequences

In [5]:
# Load sequences from FASTA files
def read_fasta(filename):
    record = SeqIO.read(filename, "fasta")
    return record.seq

reference_seq = read_fasta("reference.fna")  # replace with your reference file
target_seq = read_fasta("target.fna")  # Replace with your target file


# Sequence Alignment

In [6]:
# Initialize aligner
aligner = PairwiseAligner()
aligner.mode = 'global'  # Use global alignment (Needleman-Wunsch)
aligner.match_score = 1
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -1

# Perform alignment
alignments = aligner.align(reference_seq, target_seq)

# Print the best alignment
print(alignments[0])


target            0 GGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCAC
                  0 -----------||-------------|--||.||---||||||-|||||||.||.|||.|
query             0 -----------TT-------------G--TGTGA---CTCTGG-TAACTAGAGATCCCTC

target           60 TGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTTCAAGTAGTGTGTGCCCGTCTGTTGT
                 60 .|---|...|||.|.|.||--||--|..|...|-||.||.||||-|.|||||--------
query            30 AG---ACCACTCTAGATAG--TG--TAAAAATC-TCTAGCAGTG-GCGCCCG--------

target          120 GTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCA
                120 -----------|||-||-|--------|||----||----------||||---.|.|-.|
query            73 -----------AAC-AG-G--------GAC----TT----------GAAA---GCGA-AA

target          180 GTGGCGCCCGAACAGGGACCTGAAAGCGAAAGGGAAACCAGAGGAGCTCTCTCGACGCAG
                180 ||-------.|||||||||..|||||||||||---..||||||.||.|||||||||||||
query            94 GT-------TAACAGGGACTCGAAAGCGAAAG---TTCCAGAGAAGTTCTCTCGACGCAG

target          240 GACT

# Mutations and Insertions/Deletions

In [7]:
# list mutations and insertions/deletions

# Extract aligned sequences
aligned_ref = alignments[0][0]
aligned_target = alignments[0][1]

# Dictionaries to store mutations and insertions/deletions
mutations = {}  # Format: {position: (reference_base, target_base)}
indels = {}     # Format: {position: (reference_base, target_base)}

# Process alignment results
for i in range(len(aligned_ref)):
    ref_base = aligned_ref[i]
    target_base = aligned_target[i]

    if ref_base != target_base:
        if ref_base == "-":  # Insertion in target
            indels[i + 1] = (ref_base, target_base)
        elif target_base == "-":  # Deletion in target
            indels[i + 1] = (ref_base, target_base)
        else:  # Substitution (Mutation)
            mutations[i + 1] = (ref_base, target_base)

# Print differences
def display_changes():
  print("Mutations:")
  for key in sorted(mutations):
    print(f"Position {key}: {mutations[key][0]} → {mutations[key][1]}")

  print("\nIndels:")
  for key in sorted(indels):
    print(f"Position {key}: {indels[key][0]} → {indels[key][1]}")

def displayChangedSequence(mutatedList, start, end):
  referenceMutation = ""
  targetMutation = ""
  for i in range(len(mutatedList)):
    referenceMutation += mutatedList[i][0]
    targetMutation += mutatedList[i][1]
    # print(i)

  output = f"Position {start} to {end}: {referenceMutation} → {targetMutation}"
  print(output)
  return output


def displayChanges():
  # initialise trackers
  flag = -1
  start = None
  end = None
  prevKey = None
  mutatedList = []

  with open("mutations_output.txt", "w") as file:
    file.write("Mutations:\n")
    print("Mutations:")

    for key in sorted(mutations):
      if flag == -1:
        flag = 0
        start = key
      if prevKey != None and key - prevKey == 1:
        mutatedList.append((mutations[key][0], mutations[key][1]))
      else:
        end = prevKey
        if bool(mutatedList) == True:
          output = displayChangedSequence(mutatedList, start, end)
          file.write(output + "\n")
        start = key
        mutatedList = [(mutations[key][0], mutations[key][1])]

      prevKey = key

    flag = -1

    file.write("\nIndels:\n")
    print("\nIndels:")
    for key in sorted(indels):
      if flag == -1:
        flag = 0
        start = key
      if prevKey != None and key - prevKey == 1:
        mutatedList.append((indels[key][0], indels[key][1]))
      else:
        end = prevKey
        if bool(mutatedList) == True:
          output = displayChangedSequence(mutatedList, start, end)
          file.write(output + "\n")
        start = key
        mutatedList = [(indels[key][0], indels[key][1])]

      prevKey = key

    if mutatedList:
      output = displayChangedSequence(mutatedList, start, prevKey)
      file.write(output + "\n")

displayChanges()

Mutations:
Position 32 to 32: G → T
Position 52 to 52: G → A
Position 55 to 55: A → T
Position 59 to 59: A → T
Position 61 to 61: T → A
Position 67 to 69: AGC → CCA
Position 73 to 73: A → T
Position 75 to 75: T → G
Position 77 to 77: A → T
Position 87 to 88: TG → AA
Position 90 to 92: GTG → AAT
Position 97 to 97: A → T
Position 100 to 100: T → C
Position 107 to 107: T → C
Position 174 to 174: T → G
Position 176 to 176: T → G
Position 179 to 179: C → A
Position 190 to 190: G → T
Position 200 to 201: CT → TC
Position 216 to 217: AA → TT
Position 224 to 224: G → A
Position 227 to 227: C → T
Position 259 to 259: C → T
Position 262 to 262: G → A
Position 266 to 266: G → A
Position 280 to 280: G → A
Position 286 to 286: G → T
Position 303 to 303: A → T
Position 370 to 370: G → A
Position 379 to 380: CG → GC
Position 397 to 397: T → C
Position 399 to 399: A → G
Position 425 to 425: A → G
Position 427 to 427: T → A
Position 429 to 429: A → G
Position 436 to 436: A → T
Position 459 to 459: A → 